In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [26]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [27]:
environment = jinja2.Environment()

In [28]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [29]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [30]:
from shutil import copytree, ignore_patterns


copytree('./templates/', './app_output', ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'


'./app_output'

In [31]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [32]:
typespec

{'reasoning': 'For a gym tracking bot, we need to handle several aspects:\n1. Recording exercises with sets, reps, and weights\n2. Equipment availability for exercise suggestions\n3. Time tracking for workouts\n4. Progress tracking over time\n5. Workout routine suggestions\n\nExpected user messages:\n"I did 3 sets of bench press, 8 reps each with 185lbs"\n-> recordExercise({name: "bench press", sets: [{reps: 8, weight: 185}, ...]})\n\n"What\'s my progress on deadlifts over the last month?"\n-> getProgress({exercise: "deadlifts", timeFrame: ...})\n\n"Suggest a 45-minute workout routine for chest day. I have access to bench, dumbbells, and cables"\n-> suggestRoutine({duration: 45, equipment: ["bench", "dumbbells", "cables"], target: "chest"})',
 'typespec_definitions': 'model Exercise {\n  name: string;\n  sets: Set[];\n  notes: string;\n}\n\nmodel Set {\n  reps: integer;\n  weight: float;\n}\n\nmodel Equipment {\n  name: string;\n  type: string;\n}\n\nmodel RoutineRequest {\n  duration:

In [33]:
print(typespec["reasoning"])

For a gym tracking bot, we need to handle several aspects:
1. Recording exercises with sets, reps, and weights
2. Equipment availability for exercise suggestions
3. Time tracking for workouts
4. Progress tracking over time
5. Workout routine suggestions

Expected user messages:
"I did 3 sets of bench press, 8 reps each with 185lbs"
-> recordExercise({name: "bench press", sets: [{reps: 8, weight: 185}, ...]})

"What's my progress on deadlifts over the last month?"
-> getProgress({exercise: "deadlifts", timeFrame: ...})

"Suggest a 45-minute workout routine for chest day. I have access to bench, dumbbells, and cables"
-> suggestRoutine({duration: 45, equipment: ["bench", "dumbbells", "cables"], target: "chest"})


In [34]:
print(typespec["typespec_definitions"])

model Exercise {
  name: string;
  sets: Set[];
  notes: string;
}

model Set {
  reps: integer;
  weight: float;
}

model Equipment {
  name: string;
  type: string;
}

model RoutineRequest {
  duration: duration;
  equipment: Equipment[];
  targetMuscles: string[];
}

model Progress {
  exercise: string;
  history: WorkoutHistory[];
}

model WorkoutHistory {
  date: utcDateTime;
  sets: Set[];
  totalVolume: float;
}

interface GymBot {
  @llm_func(2)
  recordExercise(exercise: Exercise): void;

  @llm_func(3)
  getProgress(exerciseName: string, startDate: utcDateTime, endDate: utcDateTime): Progress;

  @llm_func(1)
  suggestRoutine(request: RoutineRequest): Exercise[];

  @llm_func(1)
  updateAvailableEquipment(equipment: Equipment[]): void;
}


In [35]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [36]:
print(drizzle["drizzle_schema"])

import { 
  integer, 
  pgTable, 
  text, 
  timestamp, 
  real, 
  jsonb,
  primaryKey,
  serial 
} from "drizzle-orm/pg-core";

// Equipment table
export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  type: text("type").notNull(),
});

// Exercise table
export const exerciseTable = pgTable("exercise", {
  id: serial("id").primaryKey(),
  name: text("name").notNull().unique(),
  notes: text("notes"),
});

// Sets table - normalized to store individual sets
export const setsTable = pgTable("sets", {
  id: serial("id").primaryKey(),
  exercise_id: integer("exercise_id")
    .references(() => exerciseTable.id)
    .notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight").notNull(),
});

// Workout History table
export const workoutHistoryTable = pgTable("workout_history", {
  id: serial("id").primaryKey(),
  exercise_id: integer("exercise_id")
    .references(() => exerciseTable.id)
    .notNull(),
  date: ti

In [37]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [38]:
funcs

[{'name': 'recordExercise',
  'description': 'Record details of a completed exercise including the name, sets performed (reps and weights), and any additional notes.',
  'examples': ['I did 3 sets of bench press today with 135lbs',
   'Record my squats: 5 reps at 225lbs for 3 sets',
   'Just finished deadlifts - 2 sets of 8 reps at 185lbs']},
 {'name': 'getProgress',
  'description': 'Retrieve and analyze progress for a specific exercise over a given time period, showing history of sets and total volume',
  'examples': ['Show my bench press progress for the last month',
   "What's my squat history from January to now?",
   'How have my deadlifts improved since last quarter?']},
 {'name': 'suggestRoutine',
  'description': 'Generate a workout routine based on available equipment, target muscles, and desired duration',
  'examples': ['Give me a 45-minute leg workout with dumbbells',
   'Suggest a chest and triceps routine using only bodyweight exercises',
   'Create a full body workout f

In [39]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [40]:
print(pre_response.content[0].text)

<instructions>
The updateAvailableEquipment function expects an array of Equipment objects. Each Equipment object must have:
- name: A descriptive string name of the equipment
- type: A category/type classification string

When processing user input:
1. Convert equipment mentions into properly structured Equipment objects
2. If type is not explicitly mentioned, infer from context or use general categories like "weights", "cardio", "bodyweight"
3. Multiple pieces of equipment should be grouped into an array
4. Equipment names should be normalized to standard terminology
</instructions>

<examples>
    <example>
        <input>I have dumbbells, a bench, and a pull-up bar at home</input>
        <output>{
  "equipment": [
    {
      "name": "dumbbells",
      "type": "weights"
    },
    {
      "name": "bench",
      "type": "weights"
    },
    {
      "name": "pull-up bar",
      "type": "bodyweight"
    }
  ]
}</output>
    </example>

    <example>
        <input>Just got new equipm

In [41]:
pre_processors

{'recordExercise': {'instructions': 'When handling user input for recordExercise function:\n1. Extract exercise name from the input text\n2. Parse sets information in format: weight x reps or reps@weight\n3. Notes are optional - include if provided, otherwise use empty string\n4. Multiple sets should be separated by commas or new lines\n5. Weight should be converted to float type\n6. Reps should be converted to integer type',
  'examples': [('bench press 80kg x 8 reps, 85kg x 6, 85x5',
    '{\n            "exercise": {\n                "name": "bench press",\n                "sets": [\n                    {"weight": 80.0, "reps": 8},\n                    {"weight": 85.0, "reps": 6},\n                    {"weight": 85.0, "reps": 5}\n                ],\n                "notes": ""\n            }\n        }'),
   ('squats 100x5, 120x3, 120x3 feeling tired today',
    '{\n            "exercise": {\n                "name": "squats",\n                "sets": [\n                    {"weight":

In [42]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)


In [43]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { \n  exerciseTable, \n  setsTable, \n  workoutHistoryTable,\n  workoutHistorySetsTable \n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: {\n  name: string;\n  sets: Array<{ reps: number; weight: number }>;\n  notes: string;\n}): Promise<void> => {\n  // Start a transaction since we\'re performing multiple related operations\n  await db.transaction(async (tx) => {\n    // Check if exercise exists, if not create it\n    let exerciseRecord = await tx\n      .select()\n      .from(exerciseTable)\n      .where(eq(exerciseTable.name, exercise.name))\n      .limit(1);\n\n    let exerciseId: number;\n    \n    if (exerciseRecord.length === 0) {\n      const [newExercise] = await tx\n        .insert(exerciseTable)\n        .values({\n          name: exercise.name,\n          notes: exercise.notes\n        })\n        .returning({ id: exerciseTable.id });\n      \n   

In [44]:
with open("./templates/interpolation/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [45]:
hdl_key = "recordWorkout"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

KeyError: 'recordWorkout'

In [167]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { 
    workoutSessionsTable, 
    exercisesTable, 
    sessionExercisesTable 
} from '../db/schema/application';

const handle = async (workout: WorkoutSession): Promise<void> => {
    // Insert the workout session
    const [workoutSession] = await db
        .insert(workoutSessionsTable)
        .values({
            date: workout.date,
            duration_minutes: Math.floor(workout.duration / 60), // Convert duration to minutes
        })
        .returning({ id: workoutSessionsTable.id });

    // Insert exercises and create session-exercise relationships
    for (const exercise of workout.exercises) {
        // Insert the exercise
        const [newExercise] = await db
            .insert(exercisesTable)
            .values({
                name: exercise.name,
                sets: exercise.sets,
                reps: exercise.reps,
                weight: e

In [170]:
print(handlers["planWorkout"]["handler"])

import { db } from "../db";
import { 
    workoutPlansTable, 
    planExercisesTable, 
    planEquipmentTable,
    exercisesTable,
    equipmentTable 
} from '../db/schema/application';
import { eq } from 'drizzle-orm';

const handle = async (duration: number, equipment: { name: string, category: string }[]): Promise<{
    targetDuration: number,
    exercises: Array<{
        name: string,
        sets: number,
        reps: number,
        weight: number,
        notes: string
    }>,
    equipment: Array<{
        name: string,
        category: string
    }>
}> => {
    // Create the workout plan
    const [workoutPlan] = await db
        .insert(workoutPlansTable)
        .values({
            target_duration_minutes: duration
        })
        .returning();

    // Insert or get equipment IDs
    const equipmentIds = await Promise.all(
        equipment.map(async (eq) => {
            const [existingEquipment] = await db
                .select()
                .from(equipmentT

In [50]:
import os

with open("./templates/interpolation/handler.debug.tpl", "r") as f:
    handler_debug_tpl = environment.from_string(f.read())
    
    for handler_name in handlers.keys():
        params = {
            "handler": {"name": handler_name},
            #"instructions": pre_processors[hdl_key]["instructions"],
            #"examples": pre_processors[hdl_key]["examples"],
        }
        file_content = handler_debug_tpl.render(**params)

        # Convert PascalCase to snake_case for file naming
        handler_snake_name = ''.join(['_' + c.lower() if c.isupper() else c for c in handler_name]).lstrip('_')
        handler_file_name = f'{handler_snake_name}_handler_debug'
        
        with open(os.path.join('./app_output/app_schema/src/handlers', handler_file_name + '.ts'), 'w') as f:
            f.write(file_content)
            
        handlers[handler_name]["module_name"] = handler_file_name
        
        print(f"Handler: {handler_file_name}")
        print(file_content)


Handler: record_exercise_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler recordExercise executed';
    return [{ role: 'assistant', content: content }];
};

export const recordExercise = new GenericHandler<[string], string>(handle, preProcessor, postProcessor);
Handler: get_progress_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler getProgress executed';
    return [{ role: 'assistant', 

In [51]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { \n  exerciseTable, \n  setsTable, \n  workoutHistoryTable,\n  workoutHistorySetsTable \n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: {\n  name: string;\n  sets: Array<{ reps: number; weight: number }>;\n  notes: string;\n}): Promise<void> => {\n  // Start a transaction since we\'re performing multiple related operations\n  await db.transaction(async (tx) => {\n    // Check if exercise exists, if not create it\n    let exerciseRecord = await tx\n      .select()\n      .from(exerciseTable)\n      .where(eq(exerciseTable.name, exercise.name))\n      .limit(1);\n\n    let exerciseId: number;\n    \n    if (exerciseRecord.length === 0) {\n      const [newExercise] = await tx\n        .insert(exerciseTable)\n        .values({\n          name: exercise.name,\n          notes: exercise.notes\n        })\n        .returning({ id: exerciseTable.id });\n      \n   

In [52]:
with open("./templates/interpolation/logic_index.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/index.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

import { GenericHandler } from "../common/handler";

import { recordExercise } from "../handlers/record_exercise_handler_debug";

import { getProgress } from "../handlers/get_progress_handler_debug";

import { suggestRoutine } from "../handlers/suggest_routine_handler_debug";

import { updateAvailableEquipment } from "../handlers/update_available_equipment_handler_debug";


export const handlers: {[key: string]: GenericHandler<any[], any>} = {
    
    'recordExercise': recordExercise,
    
    'getProgress': getProgress,
    
    'suggestRoutine': suggestRoutine,
    
    'updateAvailableEquipment': updateAvailableEquipment,
    
};


In [49]:
with open("./templates/interpolation/router.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/router.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

import { client } from "../common/llm";
import { Message } from "../common/handler";
const nunjucks = require("nunjucks");

const router_prompt: string = `
Based on converstation between user and assistant determine which function should
handle current message based on function description and message content.
{% for function in functions%}
<function name="{{function.name}}">
    <description>{{function.description}}</description>
    {% for example in function.examples %}
    <example>{{example}}</example>{% endfor %}
</function>
{% endfor %}
Reply with the name of the function only.

Conversation:
{% for message in messages %}
<role name="{{message.role}}">{{message.content}}</role>
{% endfor %}
`;

export interface FunctionDef {
    name: string;
    description: string;
}

const functions: FunctionDef[] = [
    
]

export const getRoute = async (messages: Message[]): Promise<string> => {
    const request = nunjucks.renderString(router_prompt, { messages, functions });
    const re

In [24]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [25]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: string;\n    ingredients: Ingredient[];\n}\n\nmodel Ingredient {\n    name: string;\n    calories: integer;\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: utcDateTime, to: utcDateTime): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [26]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [27]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [28]:
from shutil import copytree, ignore_patterns

In [29]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [30]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
